# Traffic Sign Classification - LeNet Deep Network

This project utilizes the LeNet deep network architecture to classify 42 different types of traffic signs. LeNet refers to a convolutional neural network that can be used for computer vision and classification models. This project showcases a step-by-step implementation of the model as well as in-depth notes to customize the model further for higher accuracy.

## Importing the necessary libraries

In [2]:
import pandas as pd
import numpy as np

> Classic libraries that will help us read and analyze data

In [4]:
import matplotlib.pyplot as plt
import seaborn as sns

> Libraries used for plotting and data visualization

In [5]:
import pickle

> Using pickle package to open our data, not much use after that

## Importing the dataset

Importing the data into three different sections: training, validation, and
testing. Training data is used to train the network, testing data is used to test
the trained network with data that it has never seen to see how it would perform in
the real world. The validation dataset ensures that we avoid overfitting by showing
the network validation data every epoch (or run) in a process called 
cross-validation to ensure that the network is not focusing on the details of the
training data. 

In [9]:
with open("./traffic-signs-data/train.p", mode = 'rb') as training_data:
    train = pickle.load(training_data)

In [10]:
with open("./traffic-signs-data/valid.p", mode = 'rb') as validation_data:
    valid = pickle.load(validation_data)

In [11]:
with open("./traffic-signs-data/test.p", mode = 'rb') as testing_data:
    test = pickle.load(testing_data)

### Splitting the dataset into training and testing sets

Splitting the data into our individual training, validation, and testing set variables.

In [12]:
X_train, y_train = train['features'], train['labels']

> Assigning the features of the training set as X_train and the dependent variable (labels) as y_train

In [15]:
X_validation, y_validation = valid['features'], valid['labels']

> Assigning the features of the validation set as X_validation and the dependent variable (labels) as y_validation

In [16]:
X_test, y_test = test['features'], test['labels']

> Assigning the features of the testing set as X_test and the dependent variable (labels) as y_test